Importing necessary libraries

In [36]:
import pandas as pd
import numpy as np

Setting up DataFrame.

In [37]:
nyc_crashes = pd.read_csv("original-crashes.csv")
print(nyc_crashes.shape)

(100000, 29)


Starting new DataFrame: nyc_crashes_basics
Dropping columns I deem useless for this ML-algorithm.

In [38]:
to_drop = \
    ["borough","zip_code","location","on_street_name",
     "off_street_name","cross_street_name", "number_of_pedestrians_injured",
     "number_of_pedestrians_killed", "number_of_cyclist_injured",
     "number_of_cyclist_killed", "number_of_motorist_injured",
     "number_of_motorist_killed","contributing_factor_vehicle_1",
     "contributing_factor_vehicle_2", "contributing_factor_vehicle_3",
     "contributing_factor_vehicle_4","contributing_factor_vehicle_5",
     "collision_id","vehicle_type_code1","vehicle_type_code2",
     "vehicle_type_code_3","vehicle_type_code_4","vehicle_type_code_5"]

nyc_crashes_basics = nyc_crashes.drop(columns=to_drop)

Exploring DataFrame "nyc_crashes_basics"

In [39]:
print(nyc_crashes_basics.columns)
print(nyc_crashes_basics.isna().sum())
print(nyc_crashes_basics.dtypes)

Index(['crash_date', 'crash_time', 'latitude', 'longitude',
       'number_of_persons_injured', 'number_of_persons_killed'],
      dtype='object')
crash_date                      0
crash_time                      0
latitude                     8035
longitude                    8035
number_of_persons_injured       0
number_of_persons_killed        0
dtype: int64
crash_date                    object
crash_time                    object
latitude                     float64
longitude                    float64
number_of_persons_injured      int64
number_of_persons_killed       int64
dtype: object


In [40]:
print(nyc_crashes_basics.head(1).to_string())

                crash_date crash_time   latitude  longitude  number_of_persons_injured  number_of_persons_killed
0  2017-04-18T00:00:00.000      23:10  40.536728 -74.193344                          0                         0


Feature slicing crash_date into ["day", "month", "year"]

In [41]:
# Function for the apply-method
def splitting_date(date):
    date = date.split(sep="T")
    date_lst = date[0].split(sep="-")
    day = int(date_lst[2])
    month = int(date_lst[1])
    year = int(date_lst[0])
    return day, month, year

nyc_crashes_basics["day"], nyc_crashes_basics["month"], nyc_crashes_basics["year"] \
    = zip(*nyc_crashes["crash_date"].apply(lambda x: splitting_date(x)))

# Checking if the new columns have the correct min-max
print(nyc_crashes_basics[["day","month","year"]].describe())

# Dropping "crash_date" column.
nyc_crashes_basics = nyc_crashes_basics.drop(columns=["crash_date"])

# Bringing new columns to the front
cols_to_move = ["day", "month","year"]
nyc_crashes_basics = nyc_crashes_basics[cols_to_move + [col for col in nyc_crashes_basics.columns if col not in cols_to_move]]

                 day          month           year
count  100000.000000  100000.000000  100000.000000
mean       16.336260       7.022990    2019.378560
std         8.959698       1.828326       0.779332
min         1.000000       2.000000    2013.000000
25%         8.000000       6.000000    2019.000000
50%        17.000000       7.000000    2020.000000
75%        24.000000       8.000000    2020.000000
max        31.000000      12.000000    2020.000000


"crash_time" to type: int.
Keeping just the hour.

In [42]:
nyc_crashes_basics["crash_time"] \
    = nyc_crashes_basics["crash_time"].apply(lambda x: int(x.split(sep=":")[0]))
print(nyc_crashes_basics.crash_time.value_counts())

17    7128
16    7033
14    6723
15    6302
18    6150
13    5978
12    5524
11    5041
19    4878
10    4670
9     4545
8     4405
20    4255
0     4139
21    3927
22    3639
23    3151
7     2832
6     2127
1     2024
2     1549
5     1384
3     1327
4     1269
Name: crash_time, dtype: int64


Checking for rows that contains NaN's.

In [43]:
print(nyc_crashes_basics.isna().sum())

day                             0
month                           0
year                            0
crash_time                      0
latitude                     8035
longitude                    8035
number_of_persons_injured       0
number_of_persons_killed        0
dtype: int64


Dropping rows where the columns "longitude", and "latitude" contain NaN"s.

In [44]:
nyc_crashes_basics = nyc_crashes_basics.dropna(subset=["latitude","longitude"],how="all")

Dropping rows where the columns "longitude", and "latitude" contain same values.

In [46]:
print(nyc_crashes_basics.isna().sum())
print(nyc_crashes_basics.shape)
print(nyc_crashes_basics[nyc_crashes_basics["longitude"] == 0].shape)
print(nyc_crashes_basics[nyc_crashes_basics["latitude"] == 0].shape)
nyc_crashes_basics = nyc_crashes_basics[nyc_crashes_basics["longitude"] != nyc_crashes_basics["latitude"]]
print(nyc_crashes_basics.shape)

day                          0
month                        0
year                         0
crash_time                   0
latitude                     0
longitude                    0
number_of_persons_injured    0
number_of_persons_killed     0
dtype: int64
(91965, 8)
(169, 8)
(169, 8)
(91796, 8)


In [56]:
group_long_lat = nyc_crashes_basics.groupby(["longitude","latitude"])[["number_of_persons_injured","number_of_persons_killed"]].sum()
print(group_long_lat.nlargest(columns="number_of_persons_injured", n=5))
print(group_long_lat.nlargest(columns="number_of_persons_killed", n=5))

                     number_of_persons_injured  number_of_persons_killed
longitude latitude                                                      
-73.86536 40.651863                         39                         0
-73.82226 40.664960                         39                         0
-73.91282 40.861862                         36                         0
-73.90905 40.862680                         27                         0
-73.76736 40.656160                         27                         2
                     number_of_persons_injured  number_of_persons_killed
longitude latitude                                                      
-73.90059 40.589930                          8                         3
-73.99281 40.742900                          3                         2
-73.97554 40.798060                          0                         2
-73.90091 40.811428                          1                         2
-73.89702 40.640373                          8     